In [1]:
#===============================================================================================================
# AI_HUB(https://aihub.or.kr/) 에 '대규모 웹데이터 기반 한국어 말뭉치 데이터'(json파일) 중에서
# ***라벨링된 파일를 파싱해서, 2줄짜리 txt 말뭉치 만드는 예제
# => 대규모 웹데이터 기반 한국어 말뭉치 데이터 
# : https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=624
#
# 1. 위 URL에서 대규모 웹데이터 기반 한국어 말뭉치 데이터  다운로드
# 2. 압축 풀기 
# => unzip abc.zip -d <경로> # 특정 디렉토리에 압축 해제
# 
#==============================================================================================================
import torch
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import json
import os
import sys
sys.path.append("..")
from myutils import getListOfFiles

In [2]:
# 폴더에 있는 파일 목록 얻기(*반드시 /로 끝나야함)
inpdirectory = '../../data11/ai_hub/vl1/사건사고/'
files = getListOfFiles(inpdirectory)

print('*file_count: {}, file_list:{}'.format(len(files), files[0:5]))

*file_count: 381, file_list:['../../data11/ai_hub/vl1/사건사고/BWAC217000002813.json', '../../data11/ai_hub/vl1/사건사고/BWAC217000002814.json', '../../data11/ai_hub/vl1/사건사고/BWAC217000002815.json', '../../data11/ai_hub/vl1/사건사고/BWAC217000002816.json', '../../data11/ai_hub/vl1/사건사고/BWAC217000002817.json']


In [3]:
MIN_SENTENCE_LEN = 10        # MIN_SETENCE_LEN 이하인 문장은 추가 안함
SEP_STRING = ';.;'           #2문장 구분자 => 문장A+[SET_STR]+문장B
remove_list = ['@(이메일)', '<저작권자ⓒ', '무단전재-재배포 금지']    # 문장에 remove_list에 들어간 경우 추가 안함 

# 출력 파일 열기 
outfilepath = '../../data11/ai_hub/vl1/사건사고-2줄.txt'
outfile = open(outfilepath, 'wt', encoding='UTF8')


# 해당 json 파일들을 불러옴
for idx, file in enumerate(tqdm(files)):
    if ".json" not in file:  #.json 파일이 아니면 합치지 않음
        continue
        
    # json 파일 로딩 => [named_entity] entry만 불러옴
    json_data = json.load(open(file, "r", encoding="utf-8"))['named_entity']

    for entry in json_data:
        sentences = []
        title = entry['title']
        for title_entry in title:
            tsentence = title_entry['sentence']
            
            if len(tsentence) > MIN_SENTENCE_LEN:
                # remove 패턴인 경우에는 추가안함
                isremove = 0
                for pattern in remove_list:
                    result = tsentence.find(pattern)
                    if result > 0:
                        isremove = 1
                        break
                              
                if isremove == 0:
                    sentences.append(tsentence)
                 
        content = entry['content']
        for content_entry in content:
            csentence = content_entry['sentence']
            
            if len(csentence) > MIN_SENTENCE_LEN:
                # remove 패턴인 경우에는 추가안함
                isremove = 0
                for pattern in remove_list:
                    result = csentence.find(pattern)
                    if result > 0:
                        isremove = 1
                        break
                if isremove == 0:
                    sentences.append(csentence)
                    
        num_sentences = len(sentences)
        start = 0
        
        '''
        # json 파일 별루 1문장으로 출력
        if num_sentences > 1:
            outfile.write(SEP_STRING.join(sentences))             
            outfile.write('\n')
        '''
        
        # 여러줄 문장을 2줄씩 나눔.
        if num_sentences > 1:
            data = []
            for idx in range(num_sentences):
            
                # 2문장을 '..;' 기준으로 해서 1나의 문장으로. 만듬
                data.append(sentences[start].strip() + SEP_STRING + sentences[start+1].strip())
            
                start += 2
                if start >= num_sentences-1:
                    break  
                
            outfile.write("\n".join(data))
        
        
# 출력파일 닫기
outfile.close()

  0%|          | 0/381 [00:00<?, ?it/s]

In [4]:
# out 생성된 파일 읽어오기(한줄씩 읽어오기)
with open(outfilepath, 'r', encoding='UTF8') as f:
     lines = f.readlines()
        
print(len(lines))

73074


In [7]:
print(lines[10000:10005])

['경찰이 포켓몬고 캐릭터가 대량으로 출몰하는 지역을 빠르게 수색하면서 10여분만에 발견했지만 자칫하면 아이를 잃어버릴 수도 있는 상황이었다.;.;전북대학교 학생군사교육단 부지도 포켓몬고 사용자들이 많이 찾는 장소다.\n', "군사시설 보호구역으로 일반인들의 출입해서는 안되는 장소지만 포켓몬고에 빠진 사용자들의 출입으로 골머리를 앓고있다.;.;학생군사교육단의 한 관계자는 '해당지역이 군사시설임을 들어 일반인들의 출입자제를 요청하고 매번 퇴거를 요청하고 있으나 상당수의 게임 이용자들은 개의치 않는 것 같다'고 어려움을 호소하고 있다.국립경주박물관도 포켓몬고의 캐릭터가 많이 등장하는 장소로 알려졌다.\n", "이에 국립경주박물관은 박물관 입구에 게임으로 인해 안전사고가 발생하지 않도록 주의를 당부하는 안내문을 부착하고 있다.또 캐릭터들을 자동으로 사냥해주는 프로그램까지 등장해 개인정보 유출 등에 대한 걱정도 커지고 있다.보안업계에 따르면 게임캐릭터를 자동으로 잡아주는 '오토봇(autobot)'이 비공식경로를 통해 배포돼 스마트폰의 악성파일 감염사례가 늘어나고 있다.;.;일부 검증되지 않은 불법 프로그램은 개인정보를 유출하는 문제를 일으키기도 한다.\n", "보안업계 관계자는 '불법행위로 인해 발생된 피해는 어떠한 경우에도 보상받을 수 없다.;.;반드시 정상적인 방법으로 게임을 즐기시기 바란다'고 밝혔다.<저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>동네 철학관에서 치과진료?…무면허 진료한 가짜스님 등 구속;.;(서울=뉴스1) (이름) 기자 = 동네에서 무면허로 치과진료를 한 가짜스님, 치과기공사 등이 대거 경찰에 붙잡혔다.\n", '서울 (이름)찰서는 보건범죄단속에관한특별조치법 위반, 의료기사등에관한법률 위반 혐의로 엄모씨(63)와 이모씨(62)를 구속하고 치과기공사 이모씨(52)와 박모씨(57) 등 4명을 불구속 입건했다고 25일 밝혔다.;.;경찰에 따르면 엄씨는 2012년 6월부터 지난해 12월까지 서울 구로동에 있는 철학관 안 밀실에 치과용 의자 등을